In [ ]:
import ipycytoscape
from py2neo import Graph, Node, Relationship

### Create Neo4j Graph

In [ ]:
graph = Graph(password="neo4jbinder")

In [ ]:
nicole = Node("Person", name="Nicole", age=24)
drew = Node("Person", name="Drew", age=20)

mtdew = Node("Drink", name="Mountain Dew", calories=9000)
cokezero = Node("Drink", name="Coke Zero", calories=0)

coke = Node("Manufacturer", name="Coca Cola", url="https://www.coca-cola.com/")
pepsi = Node("Manufacturer", name="Pepsi", url="https://www.pepsi.com/")

In [ ]:
graph.create(Relationship(nicole, "LIKES", cokezero))
graph.create(Relationship(nicole, "LIKES", mtdew))
graph.create(Relationship(drew, "LIKES", mtdew))
graph.create(Relationship(coke, "MAKES", cokezero))
graph.create(Relationship(pepsi, "MAKES", mtdew))

### Run query

In [ ]:
query = """
MATCH p=()-[]->() RETURN p
"""

In [ ]:
subgraph = graph.run(query).to_subgraph()

### Display graph with Cytoscape

In [ ]:
widget = ipycytoscape.CytoscapeWidget()
widget.graph.add_graph_from_neo4j(subgraph, property_summary='properties')

In [ ]:
node_style = {'selector': 'node', 'style': {'label': 'data(name)'}}
edge_style = {'selector': 'edge', 'style': {'label': 'data(name)', 'text-rotation': 'autorotate'}}
style = [node_style, edge_style]
widget.set_style(style)

In [ ]:
widget.set_tooltip_source('properties')

In [ ]:
widget